In [ ]:
import sys
from scapy.all import sniff, get_if_list, conf

# --- Interface listing / selection ---

def list_ifaces():
    ifs = get_if_list()
    for i, iface in enumerate(ifs):
        print(f"{i:2d}: {iface}")
    return ifs

print("Available interfaces:")
ifs = list_ifaces()

# Safety: live capture is disabled by default.
do_live = False
iface = None  # set to interface name or index to capture on a specific interface

# Optional interactive selection (only when running in a true terminal)
sel = ""
try:
    if sys.stdin.isatty():
        sel = input("Enter interface index or name to use (press Enter to use default/any): ").strip()
except Exception:
    sel = ""

if sel:
    try:
        idx = int(sel)
        iface = ifs[idx]
    except Exception:
        iface = sel

print("Using interface:", iface or "any (all interfaces)")
print("Note: set do_live = True and re-run to perform live capture (requires Npcap + admin on Windows).")

def packet_handler(packet):
    print(packet.summary())

if do_live:
    if not getattr(conf, 'use_pcap', False):
        print("libpcap provider not available; live capture may fail.")
    try:
        sniff(prn=packet_handler, iface=iface, count=1, timeout=10)
    except PermissionError:
        print("Permission error: run VS Code as Administrator to capture packets.")
    except Exception as e:
        print("sniff failed:", e)
else:
    print("Live sniff not executed — set do_live = True to capture.")

Ether / ARP who has 192.168.5.11 says 0.0.0.0 / Padding


# Scapy: interface selection and safe demo

- This notebook lists available interfaces and lets you select one for live capture.
- Live capture is **disabled by default**. To capture live packets:
  1. Make sure Npcap is installed and you have Administrator privileges.  
  2. Set `do_live = True` in the first cell and optionally set `iface` (name or index).  
  3. Re-run the first cell.
- Use the offline demo cell below to create & parse packets without Npcap or admin rights.

In [ ]:
from scapy.all import Ether, IP, TCP, raw

print("Offline demo — create, serialize, and parse a packet")

pkt = Ether()/IP(dst="192.0.2.1")/TCP(dport=80, sport=12345)/b"Hello, world"
print("Created packet summary:", pkt.summary())
pkt.show()

data = raw(pkt)
print("\nSerialized bytes length:", len(data))

parsed = Ether(data)
print("Parsed-from-bytes summary:", parsed.summary())
parsed.show()